In [6]:
import pandas as pd
from sys import platform as sys_pf
from cleanco import cleanco

In [7]:
df = pd.ExcelFile("ML_TASK_DATASET.xlsx",  sheet_name=None)

dfs = {sheet_name: df.parse(sheet_name)
          for sheet_name in df.sheet_names}

data = dfs['Sheet1']

# Single word tokens
single = ['llc', 'inc', 's.c']
Double = ['group corporation', 'production llc', 'electronics usa', 'international corp', 'international corporation']
static = ['sderldecv'] # for row no 13

df1 = data.copy() # prefer to work on the copy of the original data

# get the similar looking company names in one bucket and map all of them to smallest among all.

# create the dictionary that matches list of companies with the smalles among all of them

df1.sort_values('raw_title',inplace=True, ascending=True)

# sorting helped a lot

# group similar looking names in their own column containing only the first word of the raw_title
# search for this first word in the raw_titles if found then this first word is the true alias
# else i am thinking about the distance between two strings

# do both of the operations above on the sorted slices for example slice of the column similar_names based on the same name and perform operations on that.
columns = ['raw_title', 'unified_title', 'SimilarNames']
raw_titles_sorted = df1['raw_title'].values
dfop = pd.DataFrame()
writer = pd.ExcelWriter('output.xlsx')  # for writing into the excel file
similar_looking_names = []
for i in range(len(raw_titles_sorted)):
    tokens = raw_titles_sorted[i].split()
    similar_looking_names.append(tokens[0].replace(',', ""))

df1['SimilarNames'] = similar_looking_names
unique_similar_names = list(set(similar_looking_names))


In [8]:
for i in range(len(unique_similar_names)):
    dfTemp = df1.loc[df1['SimilarNames'] == unique_similar_names[i]]
    if dfTemp["raw_title"].values.__contains__(unique_similar_names[i]):
        listUnified = []
        for j in range(len(dfTemp["raw_title"].values)):
            listUnified.append(unique_similar_names[i])
        dfTemp["unified_title"] = listUnified
        dfop = dfop.append(dfTemp, ignore_index=True)

    else:
        tempValues = []

        for j in range(len(dfTemp["raw_title"].values)):
        # Iterate over each row and assign the cleaned name to the unified_title
        # aleways check for static first
            tokens = dfTemp["raw_title"].values[j].split(",")
            tokens1 = dfTemp["raw_title"].values[j].rsplit(' ')
            if single.__contains__(tokens[len(tokens) - 1].lower()):
                tempValues.append(tokens[0].strip(','))
        # first trim based on ","
        # get the last word and check if it contains in the single list
            elif Double.__contains__(tokens1[-2].lower() + " " + tokens1[-1].lower()):
                tempValues.append(tokens1[0].strip(','))
        # now trim based on " "
        # get last two words, add them and check the resultant string in Double list. if yes remove last two else
            elif single.__contains__(tokens1[-1].lower()):
                tempValues.append(tokens1[0] + " " + tokens1[1])
        # get last word check it in single list if yes remove it.
            else:
                tempValues.append(dfTemp["raw_title"].values[j])
        # the company is cleaned already
        dfTemp["unified_title"] = tempValues
        dfop = dfop.append(dfTemp, ignore_index=True)

/Users/hiteshsantwani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/hiteshsantwani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [10]:
# Adding Cleanco as the further processing step

cleancoList = []
unified = dfop['unified_title'].values
for i in range(len(unified)):
    x = cleanco(unified[i])
    cleancoList.append(x.clean_name())

dfop['unified_title'] = cleancoList    

In [11]:
dfop.drop('SimilarNames', axis=1)
writer = pd.ExcelWriter('output.xlsx')  # for writing into the excel file
dfop.to_excel(writer,'Sheet1')

writer.save() # committing the changes.